<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamA/blob/main/TeamA_Cre_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Erste Erstellung der SQL Datenbank mit allen Daten aus der Disease 'Covid19';
Import der notwendigen Packete und Funktionen; Eingabe des Pfades der Dateien

In [ ]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv

# WICHTIG: In Colab Links auf dem Ordner "Dateien" klciken und dann oben "Googel Drive bereitstellen" auswählen. DAmit stellt man die Verbindung zu seinem persönlichen Drive her.
# Der Pfad muss dann so angepasst werden, dass die Dateien im eigenen Drive Ordner gefunden werden
material_path = "/content/drive/MyDrive/M3_DM"

Auswählen der Disease & Name + Speicherort der Datenbanken festlegen

In [ ]:
!ls "/content/drive/MyDrive/M3_DM" # [zum Auslesen der Dateien und Ordner in dem angegebenen Pfad]
patient_type = "covid19"
# Der hintere Teil des Pfades bestimmt den Namen der Datenbank
DB_FILE_PATH = f"{material_path}/source_{patient_type}_test.db"
csv_path = f"{material_path}/csv_data/{patient_type}/"

DB_FILE_PATH, csv_path

Leere SQL Tabellen pro csv erstellen.
Datentyp und Key (Primary und Foreign) definieren

In [ ]:
# dictionary key: name of table, values: sql query to create table
sql_table = {}

In [ ]:
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER,
                           FOREIGN KEY (COUNTY)
                              REFERENCES county (county)
                       );"""

In [ ]:
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                       );"""

In [ ]:
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           DISPENSES INTEGER,
                           TOTALCOST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)

                       );"""

In [ ]:
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""


In [ ]:
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)

                       );"""

In [ ]:
sql_table["county"]= """CREATE TABLE IF NOT EXISTS county(
                    STATE STRING,
                    COUNTY STRING PRIMARY KEY,
                    POPULATION INTEGER,
                    AREA_sqm INTEGER,
                    DENSITY_per_sqm INTEGER,
                    SEASIDE INTEGER
                  );"""

Funktionen, die zum Import der csv Dateien nötig sind

In [ ]:
def connect_to_db(db_file):
    """
    Connect to an SQlite database, if db file does not exist it will be created
    :param db_file: absolute or relative path of db file
    :return: sqlite3 connection
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()


def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data


def insert_values_to_table(cursor, table_name, csv_file_path):

    """
    Open a csv file, store its content in a list excluding header and insert the data from the list to db table
    :param table_name: table name in the database to insert the data into
    :param csv_file_path: path of the csv file to process
    :return: None
    """

    # Read CSV file content
    values_to_insert = open_csv_file(csv_file_path)

    # Insert to table
    if len(values_to_insert) > 0:
        column_names, column_numbers = get_column_names_from_db_table(cursor, table_name)

        values_str = '?,' * column_numbers
        # print("*****", values_str, column_names, column_numbers)
        values_str = values_str[:-1]

        sql_query = 'INSERT OR REPLACE INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'


        cursor.executemany(sql_query, values_to_insert)
        # conn.commit()

        print('SQL insert process finished')
    else:
        print('Nothing to insert')

        # conn.close()

    # else:
        # print('Connection to database failed')


def get_column_names_from_db_table(sql_cursor, table_name):
    """
    Scrape the column names from a database table to a list and convert to a comma separated string, count the number
    of columns in a database table
    :param sql_cursor: sqlite cursor
    :param table_name: table name to get the column names from
    :return: a comma separated string with column names, an integer with number of columns
    """

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count


Loop durch alle csv Dateien im definierten Ordner. Ablage in der erstellten SQL DB

In [ ]:
conn = connect_to_db(DB_FILE_PATH)
if conn is not None:
        cursor = conn.cursor()
        for name in sql_table.keys():
          if name != "county":
            csv_file = csv_path + name + ".csv"
          else:
            csv_file = f"{material_path}/" + name + ".csv"
          print(name, csv_file)

          cursor.execute(sql_table[name])

          insert_values_to_table(cursor, name, csv_file)
        # conn.commit()
else:
        print('Connection to database failed')

In [ ]:
# List of tables in database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

df_pat = pd.read_sql_query("SELECT COUNT(patients.Id), county.COUNTY from patients,county WHERE patients.COUNTRY = county.county GROUP BY county.county", conn)
df_county = pd.read_sql_query("SELECT * from county", conn)
print(df_county)
print(df_pat)


In [ ]:
# Commit Befehl notwendig, damit die DB gespeichert wird. Das geschieht nicht automatisch
conn.commit()
# Close Befehl schließt die Verbindung zur DB.
# conn.close()